# Frontend Application interface to the Stabilization Models

At some point this could be converted to AWS CloudFormation but at this point understanding the infrastructure at a basic AWS SDK point is highly disirable.


## Frontend Setup


Setup the buckets associated with the application.  All the files exist either as code stored in the [project repository](https://github.com/mgrandau/hand-tracking-stabilization) or data files in the [application bucket](s3://mgrandau-springboard-capstone/frontend-application/)

Attempting to do the front end as a JAM Stack application. This will use the application to maintain current state of the application. Node.js is pre installed run the following commandlines

```bash
node -v
npm -v
```

In [1]:
import logging
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region='us-west-2'):
    try:
        s3_client = boto3.client('s3', region_name=region)
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)

    except ClientError as e:
        logging.error(e)
        return False
    return True

In [6]:
create_bucket("mgrandau-springboard-capstone-app")

True

Load the application files.

In [7]:
s3 = boto3.resource('s3')

def copy_to_application(filename):
    copy_source = {
        'Bucket': 'mgrandau-springboard-capstone',
        'Key': f'frontend-application/{filename}'
    }
    s3.meta.client.copy(copy_source, 'mgrandau-springboard-capstone-app', filename)
    object_acl = s3.ObjectAcl('mgrandau-springboard-capstone-app',filename)
    response = object_acl.put(ACL='public-read')
        
copy_to_application('index.html')
copy_to_application('stable.mp4')
copy_to_application('unstable.mp4')


This is the [photoExample](https://mgrandau-springboard-capstone-app.s3.us-west-2.amazonaws.com/photoExample.html)

## Frontend Teardown

Cleanup the application buckets

In [4]:
def delete_bucket(bucket_name):
    try:
        import boto3

        s3_client = boto3.resource('s3')
        bucket = s3_client.Bucket(bucket_name)
        bucket.objects.all().delete()
        bucket.delete()
        
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [5]:
delete_bucket("mgrandau-springboard-capstone-app")

True